In [ ]:
Рассмотреть генератор псевдослучайной последовательности. 
Вначале выбираются два больших простых числа p и q. Числа p и q должны быть оба сравнимы с 3 по модулю 4. 
Далее вычисляется число M = p* q, называемое целым числом Блюма. 

Затем выбирается другое случайное целое число х, взаимно простое с М. 
Вычисляем x_0=x^2 mod M. x_0 называется стартовым числом генератора. 

На каждом n-м шаге работы генератора вычисляется x_n=x_(n-1)^2 mod M . 
Результатом n-го шага является бит чётности числа x_n , то есть сумма по модулю 2 единиц в двоичном представлении элемента. 
Для данного генератора оценить статистические свойства при помощи следующих тестов:

In [2]:
import random
p = 22953686867719691230002707821868552601124472329079
assert p % 4 == 3
q = 95647806479275528135733781266203904794419563064407
assert q % 4 == 3
M = p * q
    
def bitfield(n):
    return [int(digit) for digit in bin(n)[2:]]

def gen():
    global x_, p, q, M
    if (x_ == -1):
        x_ = x
    x_ = (x_ * x) % M
    return sum(bitfield(x_)) % 2

In [4]:
x_ = -1
x = random.randint(1, M - 1)

In [5]:
n = 100000

In [9]:
#1
import numpy as np
from scipy.special import erfc

seq = []
for i in range(n):
    seq += [gen()]
seq = np.array(seq)
seq = seq * 2 - 1
s_obs = seq.sum() / np.sqrt(n)
print(s_obs)
p = erfc(abs(s_obs) / np.sqrt(2))
print(p)

-0.6451046426743493
0.5188593754270107


In [17]:
#2
from scipy.stats import chisquare
#m = 2
#seq = []
#for i in range(n):
#    seq += [gen()]
d = {}
for i in range(n - m + 1):
    block = ''.join(map(str, seq[i:i+m]))
    if block not in d:
        d[block] = 1
    else:
        d[block] += 1
values = np.array(list(d.values()))
chi, p = chisquare(values)
print(chi)
print(p)

1.640286402864029
0.650290035089075


In [22]:
from scipy.stats import chisquare
m = 12
d = {}
for i in range(n - m + 1):
    block = ''.join(map(str, seq[i:i+m]))
    if block not in d:
        d[block] = 1
    else:
        d[block] += 1
values = np.array(list(d.values()))
chi, p = chisquare(values)
print(chi)
print(p)

4060.6406604726526
0.6455605778890109


In [23]:
#4
seq = np.array(seq)
pi = np.mean(seq)
v = np.sum((seq[1:] + seq[:-1]) % 2) + 1
p = erfc(abs(v - 2 * n * pi * (1 - pi)) / (2 * np.sqrt(2 * n) * pi * (1 - pi)))
print(p)

0.2904963349443903


In [35]:
#5
from scipy.special import ndtr

m = 5
M = int((n - 1) / m) - 1
print(M)
pm = 1 / (M + 1) * np.sum(np.array([seq[k * m] * seq[(k + 1) * m] for k in range(M)])) - 0.25
print(pm)
sigma = np.sqrt(13 * M + 7) / (12 * (M + 1))
print(sigma)
z = pm / sigma
print(z)
p_values = ndtr(z) * 2
print(p_values)

19998
-0.002637631881594077
0.0021246200642784357
-1.241460497310078
0.2144356823993706


In [43]:
#6
def BM_algo(seq):
    seq = np.array(seq)
    N = seq.shape[0]
    s = seq[:]
    for k, x in enumerate(s):
        if x == 1:
            break
            
    f = set([k + 1, 0])
    L = k + 1
    g = set([0])
    a = k
    b = 0
    
    for n in range(k + 1, N):
        d = 0
        for i in f:
            if (i + n - 1 < N):
                d ^= s[i + n - 1]
        if d == 0:
            b += 1
        else:
            if 2 * L > n:
                f ^= set([a - b + i for i in g])
                b += 1
            else:
                tmp = f.copy()
                f = set([b - a + i for i in f]) ^ g
                L = n + 1 - L
                g = tmp.copy()
                a = b
                b = n - L + 1
    return L

print("linear span " + str(BM_algo(seq[:10000])))

linear span 4999
